Crawl dữ liệu từ shopee theo từng category

In [15]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import pandas as pd
import requests
import time



In [3]:
search_list_full = pd.read_csv('search.csv',sep='\t')
search_list_full.to_csv('search_withid.csv',sep='\t',index_label='id')
search_list_full = pd.read_csv('search_withid.csv',sep='\t')
search_list_full.head()

,id,name,category,n_sold,price,shop_address,image_url,url
0,0,Điện thoại OPPO A37 Fullbox Mới - 2GB/16GB - B...,DIENTHOAI,7800.0,1299000.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/654146756efe4b5fe7cf...,https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...
1,1,điện thoại Samsung Galaxy S7 chính hãng / full...,DIENTHOAI,6600.0,1899000.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/622be579c39efa0ea92b...,https://shopee.vn/%C4%91i%E1%BB%87n-tho%E1%BA%...
2,2,Điện thoại Nokia 1280 Nguyên zin Mới tinh chín...,DIENTHOAI,8800.0,189900.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/c92a82f0a1e8076fcc1e...,https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...
3,3,ĐIỆN THOẠI NOKIA 1202 MỚI TINH FULL SẠC PIN CH...,DIENTHOAI,6100.0,175000.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/b36a13ed46c0e3feab04...,https://shopee.vn/%C4%90I%E1%BB%86N-THO%E1%BA%...
4,4,điện thoại Samsung Galaxy S7 Edge 2 sim ram 4G...,DIENTHOAI,5400.0,2799000.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/deb114c783e8feb58d6c...,https://shopee.vn/%C4%91i%E1%BB%87n-tho%E1%BA%...


In [26]:
missing_i = []
f = open('items.csv', 'w', encoding="utf-8")
f.write(f'id\tname\tavg_rating\tn_reviews\tn_sold\tprice\tn_loved\tn_rate_5\trate_4\trate_3\trate_2\trate_1\trate_with_cmt\trate_with_imgvid\tshop_name\tshop_n_review\tshop_n_product\tshop_rate_feedback\tshop_time_feedback\tshop_age\tshop_follower\n')
for i,search_item in tqdm(search_list_full.iterrows(),total=len(search_list_full),position=0, leave=True):
    try:
        curl = search_item.url
        curl_split = curl.split('.')
        shopid = curl_split[-2]
        itemid = curl_split[-1]

        name = search_item['name']

        r = requests.get(f'https://shopee.vn/api/v2/item/get?itemid={itemid}&shopid={shopid}')
        avg_rating, n_reviews, n_loved = r.json()['item']['item_rating']['rating_star'], r.json()['item']['item_rating']['rating_count'][0],r.json()['item']['liked_count']
        
        r = requests.get(f'https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=1&itemid={itemid}&limit=6&offset=0&shopid={shopid}&type=5')
        
        rate_1, rate_2,rate_3, rate_4,rate_5 =  r.json()['data']['item_rating_summary']['rating_count']      
        rate_with_cmt =    r.json()['data']['item_rating_summary']['rcount_with_context']
        rate_with_imgvid = r.json()['data']['item_rating_summary']['rcount_with_media']
        n_sold = search_item['n_sold']
        price = search_item['price']
        cid = search_item['id']

        r = requests.get(f'https://shopee.vn/api/v2/shop/get?is_brief=1&shopid={shopid}')

        shop_name, shop_n_review , shop_n_product , shop_rate_feedback, shop_time_feedback, shop_age , shop_follower = r.json()['data']['account']['username'],r.json()['data']['rating_bad'] + r.json()['data']['rating_normal'] + r.json()['data']['rating_good'], r.json()['data']['item_count'], r.json()['data']['response_rate'], r.json()['data']['response_time'], (time.time()  - r.json()['data']['ctime']) ,r.json()['data']['follower_count']      

        
        f.write(f'{cid}\t{name}\t{avg_rating}\t{n_reviews}\t{n_sold}\t{price}\t{n_loved}\t{rate_5}\t{rate_4}\t{rate_3}\t{rate_2}\t{rate_1}\t{rate_with_cmt}\t{rate_with_imgvid}\t{shop_name}\t{shop_n_review}\t{shop_n_product}\t{shop_rate_feedback}\t{shop_time_feedback}\t{shop_age}\t{shop_follower}\n')
    except:
        # print('Missing ',i)
        missing_i.append(i)
    #     break
    # break
f.close()

  0%|          | 0/15648 [00:01<?, ?it/s]


In [6]:
missing_i

[20]

In [9]:
from cssselector import *
f = open('items.csv', 'a', encoding="utf-8")
search_list_missing = search_list_full.iloc[missing_i]
missing_i = []
for i,search_item in tqdm(search_list_missing.iterrows(),total=len(search_list_missing),position=0, leave=True):
    try:
        curl = search_item.url
        curl_split = curl.split('.')
        shopid = curl_split[-2]
        itemid = curl_split[-1]

        name = search_item['name']

        r = requests.get(f'https://shopee.vn/api/v2/item/get?itemid={itemid}&shopid={shopid}')
        avg_rating, n_reviews, n_loved = r.json()['item']['item_rating']['rating_star'], r.json()['item']['item_rating']['rating_count'][0],r.json()['item']['liked_count']
        
        r = requests.get(f'https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=1&itemid={itemid}&limit=6&offset=0&shopid={shopid}&type=5')
        
        rate_1, rate_2,rate_3, rate_4,rate_5 =  r.json()['data']['item_rating_summary']['rating_count']      
        rate_with_cmt =    r.json()['data']['item_rating_summary']['rcount_with_context']
        rate_with_imgvid = r.json()['data']['item_rating_summary']['rcount_with_media']
        n_sold = search_item['n_sold']
        price = search_item['price']
        cid = search_item['id']

        r = requests.get(f'https://shopee.vn/api/v2/shop/get?is_brief=1&shopid={shopid}')

        shop_name, shop_n_review , shop_n_product , shop_rate_feedback, shop_time_feedback, shop_age , shop_follower = r.json()['data']['account']['username'],r.json()['data']['rating_bad'] + r.json()['data']['rating_normal'] + r.json()['data']['rating_good'], r.json()['data']['item_count'], r.json()['data']['response_rate'], r.json()['data']['response_time'], (time.time()  - r.json()['data']['ctime']) ,r.json()['data']['follower_count']      

        
        f.write(f'{cid}\t{name}\t{avg_rating}\t{n_reviews}\t{n_sold}\t{price}\t{n_loved}\t{rate_5}\t{rate_4}\t{rate_3}\t{rate_2}\t{rate_1}\t{rate_with_cmt}\t{rate_with_imgvid}\t{shop_name}\t{shop_n_review}\t{shop_n_product}\t{shop_rate_feedback}\t{shop_time_feedback}\t{shop_age}\t{shop_follower}\n')
    except:
        # print('Missing ',i)
        missing_i.append(i)
    #     break
    # break
f.close()

100%|██████████| 1/1 [00:24<00:00, 24.97s/it]


In [13]:
pd.read_csv('items.csv', sep='\t').sort_values(by=['id'], ascending=True, axis=0).to_csv('items_sorted_id.csv',sep='\t',index=False)